In [ ]:
from typing import Annotated, Literal, TypedDict
import tools
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import END, START, StateGraph, MessagesState
from langgraph.prebuilt import ToolNode

In [ ]:
import re
from typing import Annotated, List, Optional
import operator

from langchain import hub
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableConfig, RunnableLambda, RunnablePassthrough, RunnableAssign
from tools import prompt_tool

class Doc(TypedDict):
    id: str
    content: str
    summary: Optional[str]
    explanation: Optional[str]
    # category: Optional[str]
class TaxonomyGenerationState(TypedDict):
    documents: List[Doc]
    summaries: List[str]
    # The raw docs; we inject summaries within them in the first step
    # Indices to be concise
    minibatches: List[List[int]]
    # Candidate Taxonomies (full trajectory)
    clusters: Annotated[List[List[dict]], operator.add]
    

# Now combine as a "map" operation in a map-reduce chain
# Input: state
# Output: state U summaries
# Processes docs in parallel
def get_input_func(state: TaxonomyGenerationState, state_input_key: str, doc_input_key: str):
    docs = state[state_input_key] # how to get the list of docs from the state (e.g. state["documents"])
    return [{doc_input_key: doc[doc_input_key]} for doc in docs] # how to get the input field from a doc (e.g. doc["content"])

def fake_llm(state):
    print(state)
    return [{"summary": "This is a summary", "explanation": "This is an explanation"} for _ in state]

def reduce_func(combined: dict, state_input_key: str, state_output_key: str):
    outputs = combined[state_output_key] # "summaries"
    documents = combined[state_input_key] # "documents"
    return {
        "documents": [
            {
                **doc,
                **{k: v for k, v in output.items()}
            }
            for doc, output in zip(documents, outputs)
        ]
    }

# map_step = RunnablePassthrough.assign(
#     # This effectively creates a "map" operation # Note you can make this more robust by handling individual errors
#     summaries=get_content | RunnableLambda(func=summary_chain.batch, afunc=summary_chain.abatch)
# )
json_schema = "{{summary: str, explanation: str}}"
summary_prompt = tools.parse_template([
    {
        "role": "system",
        "content": f"You are a summarization system. Please summarize the following text and give an explanation. Reply with the following JSON format: {json_schema}",
    },
    {
        "role": "human",
        "content": "{content}"
    }
])
summary_chain = tools.prompt_tool(
    tool_name="summarization",
    prompt_template=summary_prompt,
    model="gpt-4o-mini",
    api_key=open("../api_key").read(),
    format="json"
)

state_output_key = "summaries"
get_input = lambda x: get_input_func(x, "documents", "content")
map_step = RunnableAssign({state_output_key: get_input | RunnableLambda(func=summary_chain.batch, afunc=summary_chain.abatch)})
reduce_step = lambda x: reduce_func(x, "documents", state_output_key)
# This is actually the node itself!
map_reduce_chain = map_step | reduce_step

In [ ]:
from langgraph.graph import StateGraph, START, END

graph = StateGraph(TaxonomyGenerationState)
graph.add_node("summarize", map_reduce_chain)
graph.add_edge(START, "summarize")

app = graph.compile()

import json
docs = json.load(open("docs.json"))
final_state = app.invoke(
    {"documents": docs},
    config={"configurable": {"thread_id": 42}},
)
from pprint import pprint
pprint(final_state)


In [ ]:
from IPython.display import Image, display

try:
    display(Image(app.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

In [ ]:
from pprint import pprint
pprint(final_state)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
json_schema = "{{summary: str, explanation: str}}"
messages = [
    {
        "role": "system",
        "content": f"You are a summarization system. Please summarize the following text and give an explanation. Reply with the following JSON format: {json_schema}",
    },
    {
        "role": "human",
        "content": "{content}"
    }
]
summary_prompt = tools.parse_template(messages)
print(summary_prompt)
template = ChatPromptTemplate(summary_prompt)
template

In [ ]:
api_key = open("api_key").read()
test_steps = [
    {
        "id": "0",
        "label": "Entity Extraction",
        "state_input_key": "documents",
        "doc_input_keys": ["content"],
        "state_output_key": "entities",
        "parentIds": [],
        "execution": {
            "tool": "prompt_tool",
            "parameters": {
                "name": "entity_extraction",
                "model": "gpt-4o-mini",
                "api_key": api_key,
                "format": "json",
                "prompt_template": [
                    {
                        "role": "system",
                        "content": """
                            ** Context **
                            You are an entity extraction system. The user will give you a piece of text.
                            ** Task **
                            Your task is to extract the entities from the text. 
                            ** Requirements **
                            Reply with the following JSON format: {{ "entities": ["entity1", "entity2", ...] }}
                        """,
                    },
                    {
                        "role": "human",
                        "content": "{content}"
                    }
                ],
            }
        }
    },
    {
        "id": "1",
        "label": "Relationship Extraction",
        "state_input_key": "documents",
        "doc_input_keys": ["content", "entities"],
        "state_output_key": "relationships",
        "parentIds": ["0"],
        "execution": {
            "tool": "prompt_tool",
            "parameters": {
                "name": "relationship_extraction",
                "model": "gpt-4o-mini",
                "api_key": api_key,
                "format": "json",
                "prompt_template": [
                    {
                        "role": "system",
                        "content": """
                        ** Context **
                        You are a relationship extraction system. The user will give you a piece of text and the entities extracted from it.
                        ** Task **
                        Your task is to extract the relationships between entities from the text. 
                        ** Requirements **
                        Reply with the following JSON format: 
                            {{ "relationships": [
                                {{
                                    "label": (str, label for the relationship), 
                                    "source": (str, entity1),
                                    "target": (str, entity2),
                                    "explanation": (str, explanation of the relationship)
                                }}, 
                                {{
                                    "label": (str, label for the relationship), 
                                    "source": (str, entity1),
                                    "target": (str, entity2),
                                    "explanation": (str, explanation of the relationship)
                                }}, 
                                ...
                                ] 
                            }}
                        """
                    },
                    {
                        "role": "human",
                        "content": "Content: {content}\n Entities: {entities}"
                    }
                ]
            }
        }
    }
]
from langgraph_utils import create_graph
graph = create_graph(test_steps)

In [ ]:
from IPython.display import Image, display
app = graph.compile()
try:
    display(Image(app.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

In [ ]:
import json
app = graph.compile()
docs = json.load(open("docs.json"))
final_state = app.invoke(
    {"documents": docs},
    config={"configurable": {"thread_id": 42}},
)
from pprint import pprint
pprint(final_state.keys())
pprint(final_state)